In [ ]:
# GENYZ TEAM
# Aleta Edna Jessalyn 
# Angelina Wahyuni 
# Keisya Andretta
# Infall Syafalni

# reusing the nmos and pmos primitives from gLayout

In [16]:
import os
import subprocess

# Run a shell, source .bashrc, then printenv
cmd = 'bash -c "source ~/.bashrc && printenv"'
result = subprocess.run(cmd, shell=True, text=True, capture_output=True)
env_vars = {}
for line in result.stdout.splitlines():
    if '=' in line:
        key, value = line.split('=', 1)
        env_vars[key] = value

# Now, update os.environ with these

os.environ.update(env_vars)

import gdstk
import svgutils.transform as sg
import IPython.display
from IPython.display import clear_output
import ipywidgets as widgets

# Redirect all outputs here
hide = widgets.Output()

def display_gds(gds_file,path,scale = 3):
  
  # Generate an SVG image
  top_level_cell = gdstk.read_gds(gds_file).top_level()[0]
  top_level_cell.write_svg(os.path.join(path,'out.svg'))
    
  # Scale the image for displaying
  fig = sg.fromfile(os.path.join(path,'out.svg'))
  fig.set_size((str(float(fig.width) * scale), str(float(fig.height) * scale)))
  fig.save(os.path.join(path,'out.svg'))

  # Display the image
  IPython.display.display(IPython.display.SVG(os.path.join(path,'out.svg')))
  os.remove(os.path.join(path,'cs.gds'))

def display_component(component,path,scale = 3):
  # Save to a GDS file
  with hide:
    component.write_gds(os.path.join(path,'cs.gds'))
  display_gds(os.path.join(path,'cs.gds'),path,scale)

from glayout import MappedPDK, sky130 , gf180
#from gdsfactory.cell import cell
from gdsfactory import Component
from gdsfactory.components import text_freetype, rectangle
from glayout import nmos, pmos
from glayout import via_stack, via_array
from glayout import rename_ports_by_orientation
from glayout import tapring
from glayout.util.comp_utils import evaluate_bbox, prec_center, prec_ref_center, align_comp_to_port
from glayout.util.port_utils import add_ports_perimeter,print_ports
from glayout.util.snap_to_grid import component_snap_to_grid
from glayout.spice.netlist import Netlist
from glayout.routing.straight_route import straight_route
from glayout.routing.c_route import c_route
from glayout.routing.L_route import L_route
from glayout.primitives.mimcap import mimcap
import sys
from pathlib import Path

In [17]:
sys.path.append(os.path.abspath("../../CSWITCHv2"))

from my_CSWITCH import cswitch, add_cswitch_labels

# Activate PDK
config = {"pdk": gf180}
pdk = config["pdk"]
pdk.activate()

# Top-level cell
TOP_LEVEL = Component(name="ANALOG_TOP")
TOP_LEVEL.name = "ANALOG_TOP"

# ==================================================
# CHOPPER SWITCH A
# ==================================================
CSA1 = cswitch(pdk)
CSA2 = cswitch(pdk)
CSA3 = cswitch(pdk)
CSA4 = cswitch(pdk)

CSA1_ref = TOP_LEVEL << CSA1
CSA2_ref = TOP_LEVEL << CSA2
CSA3_ref = TOP_LEVEL << CSA3
CSA4_ref = TOP_LEVEL << CSA4

CSA1_ref.name = "CSA1"
CSA2_ref.name = "CSA2"
CSA3_ref.name = "CSA3"
CSA4_ref.name = "CSA4"

CSA2_ref.movey(CSA1_ref.ymin - 5.5)
CSA3_ref.movey(CSA2_ref.ymin - 5.5)
CSA4_ref.movey(CSA3_ref.ymin - 5.5)

Component = TOP_LEVEL.flatten()
Component.name = "CSA_ONLY"
#display_component(TOP_LEVEL, scale=0.2, path="../../")
drc_result = gf180.drc_magic(Component, Component.name)

2025-09-19 06:55:27.238 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to '/tmp/tmpzfli7lxk/CSA_ONLY$12.gds'


using default pdk_root
Defaulting to stale magic_commands.tcl

Magic 8.3 revision 528 - Compiled on Wed Jun 18 09:45:25 PM CEST 2025.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuD ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive filldiff fillpoly m1hole obsm1 fillm1 obsv1 m2hole obsm2 fillm2 obsv2 m3hole obsm3 fillm3 m4hole obsm4 fillm4 m5hole obsm5 fillm5 glass fillblock lvstext obscomment 
Scaled tech values by 10 / 1 to match internal grid scaling
Loading gf180mcuD Device Generator Menu ...
Loading "/tmp/tmpzfli7lxk/magic_commands.tcl" from command line.
Library written using GDS-II Release 6.0
Library name: library
Reading "CSA_ONLY$12".
[INFO]: Loading CSA_ONLY$12

Loading DRC CIF style.
No

In [18]:
# ==================================================
# CHOPPER SWITCH B
# ==================================================
sys.path.append(os.path.abspath("../../CSWITCHBv2"))
from my_CSWITCHB import cswitchb, add_cswitchb_labels

CSB1 = cswitchb(pdk)
CSB2 = cswitchb(pdk)

CSB1_ref = TOP_LEVEL << CSB1
CSB2_ref = TOP_LEVEL << CSB2

CSB1_ref.name = "CSB_1"
CSB2_ref.name = "CSB_2"

CSB1_ref.movex(CSA1_ref.xmax + 11)
CSB2_ref.movex(CSA1_ref.xmax + 11).movey(CSB1_ref.ymin - 5.5)

Component = TOP_LEVEL.flatten()
Component.name = "CSA_ONLY"
#display_component(TOP_LEVEL, scale=0.2, path="../../")
drc_result = gf180.drc_magic(Component, Component.name)

2025-09-19 06:55:44.032 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to '/tmp/tmpkw84bhe1/CSA_ONLY$13.gds'


using default pdk_root
Defaulting to stale magic_commands.tcl

Magic 8.3 revision 528 - Compiled on Wed Jun 18 09:45:25 PM CEST 2025.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuD ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive filldiff fillpoly m1hole obsm1 fillm1 obsv1 m2hole obsm2 fillm2 obsv2 m3hole obsm3 fillm3 m4hole obsm4 fillm4 m5hole obsm5 fillm5 glass fillblock lvstext obscomment 
Scaled tech values by 10 / 1 to match internal grid scaling
Loading gf180mcuD Device Generator Menu ...
Loading "/tmp/tmpkw84bhe1/magic_commands.tcl" from command line.
Library written using GDS-II Release 6.0
Library name: library
Reading "CSA_ONLY$13".
[INFO]: Loading CSA_ONLY$13

Loading DRC CIF style.
No

In [19]:
# ==================================================
# CONNECT CSA → CSB
# ==================================================
TOP_LEVEL << L_route(pdk, CSA1_ref.ports["OUTP_top_met_W"], CSB1_ref.ports["VINP1_top_met_N"])
TOP_LEVEL << L_route(pdk, CSA1_ref.ports["OUTN_top_met_W"], CSB1_ref.ports["VINN1_top_met_N"])
TOP_LEVEL << L_route(pdk, CSA2_ref.ports["OUTP_top_met_W"], CSB1_ref.ports["VINP2_top_met_N"])
TOP_LEVEL << L_route(pdk, CSA2_ref.ports["OUTN_top_met_W"], CSB1_ref.ports["VINN2_top_met_N"])
TOP_LEVEL << L_route(pdk, CSA3_ref.ports["OUTP_top_met_W"], CSB2_ref.ports["VINP1_top_met_N"])
TOP_LEVEL << L_route(pdk, CSA3_ref.ports["OUTN_top_met_W"], CSB2_ref.ports["VINN1_top_met_N"])
TOP_LEVEL << L_route(pdk, CSA4_ref.ports["OUTP_top_met_W"], CSB2_ref.ports["VINP2_top_met_N"])
TOP_LEVEL << L_route(pdk, CSA4_ref.ports["OUTN_top_met_W"], CSB2_ref.ports["VINN2_top_met_N"])

Component = TOP_LEVEL.flatten()
Component.name = "CSA_ONLY"
#display_component(TOP_LEVEL, scale=0.2, path="../../")
drc_result = gf180.drc_magic(Component, Component.name)

2025-09-19 06:55:45.313 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to '/tmp/tmp_msha2eb/CSA_ONLY$14.gds'


using default pdk_root
Defaulting to stale magic_commands.tcl

Magic 8.3 revision 528 - Compiled on Wed Jun 18 09:45:25 PM CEST 2025.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuD ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive filldiff fillpoly m1hole obsm1 fillm1 obsv1 m2hole obsm2 fillm2 obsv2 m3hole obsm3 fillm3 m4hole obsm4 fillm4 m5hole obsm5 fillm5 glass fillblock lvstext obscomment 
Scaled tech values by 10 / 1 to match internal grid scaling
Loading gf180mcuD Device Generator Menu ...
Loading "/tmp/tmp_msha2eb/magic_commands.tcl" from command line.
Library written using GDS-II Release 6.0
Library name: library
Reading "CSA_ONLY$14".
[INFO]: Loading CSA_ONLY$14

Loading DRC CIF style.
No

In [20]:
# ==================================================
# CHOPPER SWITCH C
# ==================================================
sys.path.append(os.path.abspath("../../CSWITCHCv2"))
from my_CSWITCHC import cswitchc, add_cswitchc_labels

CSC1 = cswitchc(pdk)
CSC1_ref = TOP_LEVEL << CSC1
CSC1_ref.name = "CSC_1"

# Place CSC after CSB (for example)
CSC1_ref.movex(CSB1_ref.xmax + 13)

# ==================================================
# CONNECT CSB -> CSC
# ==================================================
TOP_LEVEL << L_route(pdk, CSB1_ref.ports["VOUTP1_top_met_W"], CSC1_ref.ports["VINP1_top_met_N"])
TOP_LEVEL << L_route(pdk, CSB1_ref.ports["VOUTN1_top_met_W"], CSC1_ref.ports["VINN1_top_met_N"])
TOP_LEVEL << L_route(pdk, CSB1_ref.ports["VOUTP2_top_met_W"], CSC1_ref.ports["VINP2_top_met_N"])
TOP_LEVEL << L_route(pdk, CSB1_ref.ports["VOUTN2_top_met_W"], CSC1_ref.ports["VINN2_top_met_N"])
TOP_LEVEL << L_route(pdk, CSB2_ref.ports["VOUTP1_top_met_W"], CSC1_ref.ports["VINP3_top_met_N"])
TOP_LEVEL << L_route(pdk, CSB2_ref.ports["VOUTN1_top_met_W"], CSC1_ref.ports["VINN3_top_met_N"])
TOP_LEVEL << L_route(pdk, CSB2_ref.ports["VOUTP2_top_met_W"], CSC1_ref.ports["VINP4_top_met_N"])
TOP_LEVEL << L_route(pdk, CSB2_ref.ports["VOUTN2_top_met_W"], CSC1_ref.ports["VINN4_top_met_N"])

Component = TOP_LEVEL.flatten()
Component.name = "CSA_ONLY"
#display_component(TOP_LEVEL, scale=0.2, path="../../")
drc_result = gf180.drc_magic(Component, Component.name)

2025-09-19 06:55:58.215 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to '/tmp/tmpxasuy1by/CSA_ONLY$15.gds'


using default pdk_root
Defaulting to stale magic_commands.tcl

Magic 8.3 revision 528 - Compiled on Wed Jun 18 09:45:25 PM CEST 2025.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuD ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive filldiff fillpoly m1hole obsm1 fillm1 obsv1 m2hole obsm2 fillm2 obsv2 m3hole obsm3 fillm3 m4hole obsm4 fillm4 m5hole obsm5 fillm5 glass fillblock lvstext obscomment 
Scaled tech values by 10 / 1 to match internal grid scaling
Loading gf180mcuD Device Generator Menu ...
Loading "/tmp/tmpxasuy1by/magic_commands.tcl" from command line.
Library written using GDS-II Release 6.0
Library name: library
Reading "CSA_ONLY$15".
[INFO]: Loading CSA_ONLY$15

Loading DRC CIF style.
No

In [21]:
# ==================================================
# INA SECTION
# ==================================================
sys.path.append(os.path.abspath("../../INA"))

from my_INA import ina, add_ina_labels

INA1 = ina(gf180)
INA2 = ina(gf180)
INA3 = ina(gf180)
INA4 = ina(gf180)

INA1_ref = TOP_LEVEL << INA1
INA2_ref = TOP_LEVEL << INA2
INA3_ref = TOP_LEVEL << INA3
INA4_ref = TOP_LEVEL << INA4

INA1_ref.name = "INA_11114"
INA2_ref.name = "INA_21114"
INA3_ref.name = "INA_31114"
INA4_ref.name = "INA_41114"

INA1_ref.movex(CSC1_ref.xmax + 20).movey(CSC1_ref.ymax + 22)
INA2_ref.movex(CSC1_ref.xmax + 20).movey(INA1_ref.ymin - 20)
INA3_ref.movex(CSC1_ref.xmax + 20).movey(INA2_ref.ymin - 20)
INA4_ref.movex(CSC1_ref.xmax + 20).movey(INA3_ref.ymin - 20)

# ==================================================
# INA ROUTING FROM CSC
# ==================================================
TOP_LEVEL << L_route(pdk, CSC1_ref.ports["VOUTP1_top_met_W"], INA1_ref.ports["INP_top_met_N"])
TOP_LEVEL << L_route(pdk, CSC1_ref.ports["VOUTN1_top_met_N"], INA1_ref.ports["INN_top_met_W"])

TOP_LEVEL << L_route(pdk, CSC1_ref.ports["VOUTP2_top_met_W"], INA2_ref.ports["INP_top_met_N"])
TOP_LEVEL << L_route(pdk, CSC1_ref.ports["VOUTN2_top_met_N"], INA2_ref.ports["INN_top_met_W"])

TOP_LEVEL << L_route(pdk, CSC1_ref.ports["VOUTP3_top_met_W"], INA3_ref.ports["INP_top_met_N"])
TOP_LEVEL << L_route(pdk, CSC1_ref.ports["VOUTN3_top_met_N"], INA3_ref.ports["INN_top_met_W"])

TOP_LEVEL << L_route(pdk, CSC1_ref.ports["VOUTP4_top_met_W"], INA4_ref.ports["INP_top_met_N"])
TOP_LEVEL << L_route(pdk, CSC1_ref.ports["VOUTN4_top_met_N"], INA4_ref.ports["INN_top_met_W"])

Component = TOP_LEVEL.flatten()
Component.name = "CSA_ONLY"
#display_component(TOP_LEVEL, scale=0.2, path="../../")
drc_result = gf180.drc_magic(Component, Component.name)

2025-09-19 06:57:02.575 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to '/tmp/tmptkjwecz8/CSA_ONLY$16.gds'


using default pdk_root
Defaulting to stale magic_commands.tcl

Magic 8.3 revision 528 - Compiled on Wed Jun 18 09:45:25 PM CEST 2025.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuD ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive filldiff fillpoly m1hole obsm1 fillm1 obsv1 m2hole obsm2 fillm2 obsv2 m3hole obsm3 fillm3 m4hole obsm4 fillm4 m5hole obsm5 fillm5 glass fillblock lvstext obscomment 
Scaled tech values by 10 / 1 to match internal grid scaling
Loading gf180mcuD Device Generator Menu ...
Loading "/tmp/tmptkjwecz8/magic_commands.tcl" from command line.
Library written using GDS-II Release 6.0
Library name: library
Reading "CSA_ONLY$16".
[INFO]: Loading CSA_ONLY$16

Loading DRC CIF style.
No

In [22]:
# ==================================================
# CHOPPER SWITCH C
# ==================================================
sys.path.append(os.path.abspath("../../CSWITCHC"))
from my_CSWITCHC import cswitchc, add_cswitchc_labels

CSC2 = cswitchc(pdk)
CSC2_ref = TOP_LEVEL << CSC2
CSC2_ref.name = "CSC_2"
CSC2_ref.movex(INA1_ref.xmax + 11)

# ==================================================
# INA ROUTING TO CSC
# ==================================================
TOP_LEVEL << L_route(pdk, INA1_ref.ports["OUTP_top_met_W"], CSC2_ref.ports["VINP1_top_met_N"])
TOP_LEVEL << L_route(pdk, INA1_ref.ports["OUTN_top_met_N"], CSC2_ref.ports["VINN1_top_met_W"])

TOP_LEVEL << L_route(pdk, INA2_ref.ports["OUTP_top_met_W"], CSC2_ref.ports["VINP2_top_met_N"])
TOP_LEVEL << L_route(pdk, INA2_ref.ports["OUTN_top_met_N"], CSC2_ref.ports["VINN2_top_met_W"])

TOP_LEVEL << L_route(pdk, INA3_ref.ports["OUTP_top_met_W"], CSC2_ref.ports["VINP3_top_met_N"])
TOP_LEVEL << L_route(pdk, INA3_ref.ports["OUTN_top_met_N"], CSC2_ref.ports["VINN3_top_met_W"])

TOP_LEVEL << L_route(pdk, INA4_ref.ports["OUTP_top_met_W"], CSC2_ref.ports["VINP4_top_met_N"])
TOP_LEVEL << L_route(pdk, INA4_ref.ports["OUTN_top_met_N"], CSC2_ref.ports["VINN4_top_met_W"])

Component = TOP_LEVEL.flatten()
Component.name = "CSA_ONLY"
#display_component(TOP_LEVEL, scale=0.2, path="../../")
drc_result = gf180.drc_magic(Component, Component.name)

2025-09-19 06:57:15.810 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to '/tmp/tmpwrv21rl5/CSA_ONLY$17.gds'


using default pdk_root
Defaulting to stale magic_commands.tcl

Magic 8.3 revision 528 - Compiled on Wed Jun 18 09:45:25 PM CEST 2025.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuD ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive filldiff fillpoly m1hole obsm1 fillm1 obsv1 m2hole obsm2 fillm2 obsv2 m3hole obsm3 fillm3 m4hole obsm4 fillm4 m5hole obsm5 fillm5 glass fillblock lvstext obscomment 
Scaled tech values by 10 / 1 to match internal grid scaling
Loading gf180mcuD Device Generator Menu ...
Loading "/tmp/tmpwrv21rl5/magic_commands.tcl" from command line.
Library written using GDS-II Release 6.0
Library name: library
Reading "CSA_ONLY$17".
[INFO]: Loading CSA_ONLY$17

Loading DRC CIF style.
No

In [23]:
sys.path.append(os.path.abspath("../../CSWITCHB"))
from my_CSWITCHB import cswitchb, add_cswitchb_labels

CSB3 = cswitchb(pdk)
CSB4 = cswitchb(pdk)

CSB3_ref = TOP_LEVEL << CSB3
CSB4_ref = TOP_LEVEL << CSB4

CSB3_ref.name = "CSB_3"
CSB4_ref.name = "CSB_4"

CSB3_ref.movex(CSC2_ref.xmax + 11)
CSB4_ref.movex(CSC2_ref.xmax + 11).movey(CSB3_ref.ymin-5.5)

# ==================================================
# CONNECT CSB -> CSC
# ==================================================
TOP_LEVEL << L_route(pdk, CSC2_ref.ports["VOUTP1_top_met_W"], CSB3_ref.ports["VINP1_top_met_N"])
TOP_LEVEL << L_route(pdk, CSC2_ref.ports["VOUTN1_top_met_W"], CSB3_ref.ports["VINN1_top_met_N"])
TOP_LEVEL << L_route(pdk, CSC2_ref.ports["VOUTP2_top_met_W"], CSB3_ref.ports["VINP2_top_met_N"])
TOP_LEVEL << L_route(pdk, CSC2_ref.ports["VOUTN2_top_met_W"], CSB3_ref.ports["VINN2_top_met_N"])
TOP_LEVEL << L_route(pdk, CSC2_ref.ports["VOUTP3_top_met_W"], CSB4_ref.ports["VINP1_top_met_N"])
TOP_LEVEL << L_route(pdk, CSC2_ref.ports["VOUTN3_top_met_W"], CSB4_ref.ports["VINN1_top_met_N"])
TOP_LEVEL << L_route(pdk, CSC2_ref.ports["VOUTP4_top_met_W"], CSB4_ref.ports["VINP2_top_met_N"])
TOP_LEVEL << L_route(pdk, CSC2_ref.ports["VOUTN4_top_met_W"], CSB4_ref.ports["VINN2_top_met_N"])


Component = TOP_LEVEL.flatten()
Component.name = "CSA_ONLY"
#display_component(TOP_LEVEL, scale=0.2, path="../../")
drc_result = gf180.drc_magic(Component, Component.name)

2025-09-19 06:57:35.327 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to '/tmp/tmp1si4bh2f/CSA_ONLY$18.gds'


using default pdk_root
Defaulting to stale magic_commands.tcl

Magic 8.3 revision 528 - Compiled on Wed Jun 18 09:45:25 PM CEST 2025.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuD ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive filldiff fillpoly m1hole obsm1 fillm1 obsv1 m2hole obsm2 fillm2 obsv2 m3hole obsm3 fillm3 m4hole obsm4 fillm4 m5hole obsm5 fillm5 glass fillblock lvstext obscomment 
Scaled tech values by 10 / 1 to match internal grid scaling
Loading gf180mcuD Device Generator Menu ...
Loading "/tmp/tmp1si4bh2f/magic_commands.tcl" from command line.
Library written using GDS-II Release 6.0
Library name: library
Reading "CSA_ONLY$18".
[INFO]: Loading CSA_ONLY$18

Loading DRC CIF style.
No

In [24]:
# ==================================================
# CHOPPER SWITCH A
# ==================================================
CSA5 = cswitch(pdk)
CSA6 = cswitch(pdk)
CSA7 = cswitch(pdk)
CSA8 = cswitch(pdk)

CSA5_ref = TOP_LEVEL << CSA1
CSA6_ref = TOP_LEVEL << CSA2
CSA7_ref = TOP_LEVEL << CSA3
CSA8_ref = TOP_LEVEL << CSA4

CSA5_ref.name = "CSA5"
CSA6_ref.name = "CSA6"
CSA7_ref.name = "CSA7"
CSA8_ref.name = "CSA8"

CSA5_ref.movex(CSB3_ref.xmax + 8)
CSA6_ref.movey(CSA5_ref.ymin - 5.5).movex(CSB3_ref.xmax + 7)
CSA7_ref.movey(CSA6_ref.ymin - 5.5).movex(CSB3_ref.xmax + 7)
CSA8_ref.movey(CSA7_ref.ymin - 5.5).movex(CSB3_ref.xmax + 7)

# ==================================================
# CONNECT CSB → CSA
# ==================================================
TOP_LEVEL << L_route(pdk, CSA5_ref.ports["INP_top_met_W"], CSB3_ref.ports["VOUTP1_top_met_N"])
TOP_LEVEL << L_route(pdk, CSA5_ref.ports["INN_top_met_W"], CSB3_ref.ports["VOUTN1_top_met_N"])
TOP_LEVEL << L_route(pdk, CSA6_ref.ports["INP_top_met_W"], CSB3_ref.ports["VOUTP2_top_met_N"])
TOP_LEVEL << L_route(pdk, CSA6_ref.ports["INN_top_met_W"], CSB3_ref.ports["VOUTN2_top_met_N"])
TOP_LEVEL << L_route(pdk, CSA7_ref.ports["INP_top_met_W"], CSB4_ref.ports["VOUTP1_top_met_N"])
TOP_LEVEL << L_route(pdk, CSA7_ref.ports["INN_top_met_W"], CSB4_ref.ports["VOUTN1_top_met_N"])
TOP_LEVEL << L_route(pdk, CSA8_ref.ports["INP_top_met_W"], CSB4_ref.ports["VOUTP2_top_met_N"])
TOP_LEVEL << L_route(pdk, CSA8_ref.ports["INN_top_met_W"], CSB4_ref.ports["VOUTN2_top_met_N"])

Component = TOP_LEVEL.flatten()
Component.name = "CSA_ONLY"
#display_component(TOP_LEVEL, scale=0.2, path="../../")
drc_result = gf180.drc_magic(Component, Component.name)

2025-09-19 06:58:03.814 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to '/tmp/tmp05dxt4pa/CSA_ONLY$19.gds'


using default pdk_root
Defaulting to stale magic_commands.tcl

Magic 8.3 revision 528 - Compiled on Wed Jun 18 09:45:25 PM CEST 2025.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuD ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive filldiff fillpoly m1hole obsm1 fillm1 obsv1 m2hole obsm2 fillm2 obsv2 m3hole obsm3 fillm3 m4hole obsm4 fillm4 m5hole obsm5 fillm5 glass fillblock lvstext obscomment 
Scaled tech values by 10 / 1 to match internal grid scaling
Loading gf180mcuD Device Generator Menu ...
Loading "/tmp/tmp05dxt4pa/magic_commands.tcl" from command line.
Library written using GDS-II Release 6.0
Library name: library
Reading "CSA_ONLY$19".
[INFO]: Loading CSA_ONLY$19

Loading DRC CIF style.
No

In [25]:
sys.path.append(os.path.abspath("../../LPF"))

from my_LPF import LPF, add_LPF_labels

LPF1 = LPF(gf180)
LPF2 = LPF(gf180)
LPF3 = LPF(gf180)
LPF4 = LPF(gf180)

LPF1_ref = TOP_LEVEL << LPF1
LPF2_ref = TOP_LEVEL << LPF2
LPF3_ref = TOP_LEVEL << LPF3
LPF4_ref = TOP_LEVEL << LPF4

LPF1_ref.name = "LPF_1"
LPF2_ref.name = "LPF_2"
LPF3_ref.name = "LPF_3"
LPF4_ref.name = "LPF_4"

LPF1_ref.movex(CSA5_ref.xmax + 12).movey(CSC1_ref.ymax + 30)
LPF2_ref.movex(CSA6_ref.xmax + 12).movey(LPF1_ref.ymin - 15)
LPF3_ref.movex(CSA7_ref.xmax + 12).movey(LPF2_ref.ymin - 15)
LPF4_ref.movex(CSA8_ref.xmax + 12).movey(LPF3_ref.ymin - 15)

# ==================================================
# ROUTE CSA TO LPF
# ==================================================
TOP_LEVEL << L_route(pdk, CSA5_ref.ports["OUTP_bottom_met_N"],LPF1_ref.ports["VINP_top_met_W"])
TOP_LEVEL << L_route(pdk, CSA5_ref.ports["OUTN_bottom_met_W"],LPF1_ref.ports["VINN_top_met_N"])

TOP_LEVEL << L_route(pdk, CSA6_ref.ports["OUTP_bottom_met_N"],LPF2_ref.ports["VINP_top_met_W"])
TOP_LEVEL << L_route(pdk, CSA6_ref.ports["OUTN_bottom_met_W"],LPF2_ref.ports["VINN_top_met_N"])

TOP_LEVEL << L_route(pdk, CSA7_ref.ports["OUTP_bottom_met_N"],LPF3_ref.ports["VINP_top_met_W"])
TOP_LEVEL << L_route(pdk, CSA7_ref.ports["OUTN_bottom_met_W"],LPF3_ref.ports["VINN_top_met_N"])

TOP_LEVEL << L_route(pdk, CSA8_ref.ports["OUTP_bottom_met_N"],LPF4_ref.ports["VINP_top_met_W"])
TOP_LEVEL << L_route(pdk, CSA8_ref.ports["OUTN_bottom_met_W"],LPF4_ref.ports["VINN_top_met_N"])

Component = TOP_LEVEL.flatten()
Component.name = "CSA_ONLY"
#display_component(TOP_LEVEL, scale=0.2, path="../../")
drc_result = gf180.drc_magic(Component, Component.name)
#Component.write_gds("TEST_A.gds")

2025-09-19 06:59:57.343 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to '/tmp/tmp3en1v0ap/CSA_ONLY$20.gds'


using default pdk_root
Defaulting to stale magic_commands.tcl

Magic 8.3 revision 528 - Compiled on Wed Jun 18 09:45:25 PM CEST 2025.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuD ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive filldiff fillpoly m1hole obsm1 fillm1 obsv1 m2hole obsm2 fillm2 obsv2 m3hole obsm3 fillm3 m4hole obsm4 fillm4 m5hole obsm5 fillm5 glass fillblock lvstext obscomment 
Scaled tech values by 10 / 1 to match internal grid scaling
Loading gf180mcuD Device Generator Menu ...
Loading "/tmp/tmp3en1v0ap/magic_commands.tcl" from command line.
Library written using GDS-II Release 6.0
Library name: library
Reading "CSA_ONLY$20".
[INFO]: Loading CSA_ONLY$20

Loading DRC CIF style.
No

In [26]:
sys.path.append(os.path.abspath("../../CLK_CS_CAD"))

from my_CLK_CS_CAD import clkcs1, add_clkcs1_labels

CLK_CS_1 = clkcs1(gf180)
CLK_CS_2 = clkcs1(gf180)
CLK_CS_3 = clkcs1(gf180)
CLK_CS_4 = clkcs1(gf180)
CLK_CS_5 = clkcs1(gf180)
CLK_CS_6 = clkcs1(gf180)

CLK_CS_1_ref = TOP_LEVEL << CLK_CS_1
CLK_CS_2_ref = TOP_LEVEL << CLK_CS_2
CLK_CS_3_ref = TOP_LEVEL << CLK_CS_3
CLK_CS_4_ref = TOP_LEVEL << CLK_CS_4
CLK_CS_5_ref = TOP_LEVEL << CLK_CS_5
CLK_CS_6_ref = TOP_LEVEL << CLK_CS_6

CLK_CS_1_ref.name = "CLK_CS_1"
CLK_CS_2_ref.name = "CLK_CS_2"
CLK_CS_3_ref.name = "CLK_CS_3"
CLK_CS_4_ref.name = "CLK_CS_4"
CLK_CS_5_ref.name = "CLK_CS_5"
CLK_CS_6_ref.name = "CLK_CS_6"

x_clk_cs = CSA4_ref.xmin
y_clk_cs = CSA4_ref.ymin
CLK_CS_1_ref.movey(y_clk_cs - 10)
CLK_CS_2_ref.movex(CLK_CS_1_ref.xmax + 10).movey(y_clk_cs - 10)
CLK_CS_3_ref.movex(CLK_CS_1_ref.xmax + 10).movey(CLK_CS_2_ref.ymin - 8)
CLK_CS_4_ref.movex(INA1_ref.xmax + 10).movey(y_clk_cs - 10)
CLK_CS_5_ref.movex(CLK_CS_4_ref.xmax + 10).movey(y_clk_cs - 10)
CLK_CS_6_ref.movex(INA1_ref.xmax + 10).movey(CLK_CS_2_ref.ymin - 8)

Component = TOP_LEVEL.flatten()
Component.name = "CSA_ONLY"
#display_component(TOP_LEVEL, scale=0.2, path="../../")
drc_result = gf180.drc_magic(Component, Component.name)

2025-09-19 07:00:40.900 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to '/tmp/tmp7vz6fmfv/CSA_ONLY$21.gds'


using default pdk_root
Defaulting to stale magic_commands.tcl

Magic 8.3 revision 528 - Compiled on Wed Jun 18 09:45:25 PM CEST 2025.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuD ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive filldiff fillpoly m1hole obsm1 fillm1 obsv1 m2hole obsm2 fillm2 obsv2 m3hole obsm3 fillm3 m4hole obsm4 fillm4 m5hole obsm5 fillm5 glass fillblock lvstext obscomment 
Scaled tech values by 10 / 1 to match internal grid scaling
Loading gf180mcuD Device Generator Menu ...
Loading "/tmp/tmp7vz6fmfv/magic_commands.tcl" from command line.
Library written using GDS-II Release 6.0
Library name: library
Reading "CSA_ONLY$21".
[INFO]: Loading CSA_ONLY$21

Loading DRC CIF style.
No

In [27]:
sys.path.append(os.path.abspath("../../CLK_LPF"))

from my_CLKLPF import clklpf, add_clklpf_labels

CLKLPF1 = clklpf(gf180)
CLKLPF1_ref = TOP_LEVEL << CLKLPF1
CLKLPF1_ref.name = "CLK_LPF_111"
CLKLPF1_ref.movey(CLK_CS_2_ref.ymin - 19).movex(CLK_CS_4_ref.xmax + 10)

Component = TOP_LEVEL.flatten()
Component.name = "CSA_ONLY"
drc_result = gf180.drc_magic(Component, Component.name)
Component.write_gds("TEST.gds")
#display_component(TOP_LEVEL, scale=0.2, path="../../")

2025-09-19 07:00:50.773 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to '/tmp/tmpr1rr7b6o/CSA_ONLY$22.gds'


using default pdk_root
Defaulting to stale magic_commands.tcl

Magic 8.3 revision 528 - Compiled on Wed Jun 18 09:45:25 PM CEST 2025.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuD ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive filldiff fillpoly m1hole obsm1 fillm1 obsv1 m2hole obsm2 fillm2 obsv2 m3hole obsm3 fillm3 m4hole obsm4 fillm4 m5hole obsm5 fillm5 glass fillblock lvstext obscomment 
Scaled tech values by 10 / 1 to match internal grid scaling
Loading gf180mcuD Device Generator Menu ...
Loading "/tmp/tmpr1rr7b6o/magic_commands.tcl" from command line.
Library written using GDS-II Release 6.0
Library name: library
Reading "CSA_ONLY$22".
[INFO]: Loading CSA_ONLY$22

Loading DRC CIF style.
No

2025-09-19 07:00:53.064 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to 'TEST.gds'


PosixPath('TEST.gds')